## Assignment 3 - Competitive Assignment
An explanation this assignment could be found in the .pdf explanation document

### Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [3]:
# imports for reading and writing (input & output) files:
import pandas as pd
import numpy as np
import openpyxl
import os, xlrd, csv 
import re
import nltk
import string

from nltk.corpus import stopwords
import matplotlib.pyplot as plt

In [4]:
### add whatever imports you need
# YOUR CODE HERE

# ML Packages
from sklearn import preprocessing
from sklearn import metrics

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


In [6]:
def myName():
    return "Almog Menashe"

def myId():
    return 123

student_name = myName()
student_id = myId()
print ("your name is: %s" %(student_name))
print ("your id is: %d" %(student_id))

your name is: Almog Menashe
your id is: 123


### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [4]:
train_filename = '.' + os.sep + 'input' + os.sep + 'annotated_corpus_for_train.xlsx'
test_filename  = '.' + os.sep + 'input' + os.sep + 'corpus_for_test.xlsx'
df_train = pd.read_excel(train_filename, 'corpus', index_col=None, na_values=['NA'])
df_test  = pd.read_excel(test_filename,  'corpus', index_col=None, na_values=['NA'])

In [5]:
df_train.head(8)

,story,gender
0,"בוקר אחד קמתי סהרורי יצאתי מהמיטה קצת מטושטש ,...",m
1,לחבר שלי היה יום הולדת וחיפשנו מה אפשר לעשות ל...,m
2,"השנה האחרונה הייתה שנת קורונה, שנה לא פשוטה בק...",m
3,"לפני כחצי שנה עברתי לגור בצפון עם בת זוגתי, עב...",m
4,"יום חמישי רגיל, תמיד מתחיל לעבור טיפה מאוחר יו...",m
5,בשנה האחרונה חוויתי את מגפת הקורונה שהכריח את ...,m
6,בסמסטר קודם אני וכמה חברים ללימודים קבענו להיפ...,m
7,בשנה האחרונה למרות שלא היו יותר מידיי דברים לע...,f


In [6]:
df_test.head(8)

,test_example_id,story
0,0,כחלק ממסגרת ההתנדבות שלי במגלה אני הולך לפעמיי...
1,1,"לפני שנה החלטתי שאני רוצה להיות טייס, התחלתי ל..."
2,2,"בתקופת הקורונה של תחילת החיסונים נגד קורונה, א..."
3,3,כפי שכולם מכירים או שמעו מחברים עולם הדייטים ה...
4,4,"אחת החוויות שהכי זכורות לי, זו החוויה בפרו בטי..."
5,5,"ראיתי בקורונה את עולמי נופל בין ידי , בת זוגתי..."
6,6,"באחת תקופות המבחנים שהיו השנה , קראתי לא נכון ..."
7,7,"כשהשתחררתי מהצבא בגיל 22 ישר התחלתי לעבוד, הצב..."


In [7]:
#  #Import stop words from file
stpwrdpath =  '.' + os.sep + 'input' + os.sep + 'stop_word_list_heb_example.txt'
with open(stpwrdpath, 'rb') as fp:
         stopword = fp.read (). decode ('utf-8')
        
list_hebrow_stopwords = stopword.splitlines()
print(list_hebrow_stopwords)        

['לא', 'את', 'זה', 'של', 'על', 'אני', 'מה', 'עם', 'הוא', 'כל', 'אבל', 'גם', 'לי', 'יש', 'אם', 'אתה', 'רק', 'או', 'היה', 'אין', 'יותר', 'אז', 'היא', 'כי', 'לך', 'טוב', 'הם', 'מי', 'כמו', 'שלא', 'כבר', 'עוד', 'שלי', 'אחד', 'הזה', 'למה', 'לו', 'ולא', 'כמה', 'היום', 'אותו', 'איך', 'כן', 'כך', 'שהוא', 'צריך', 'שלך', 'להיות', 'עד', 'עכשיו', 'שם', 'שלו', 'שאני', 'ממש', 'שזה', 'כדי', 'אחרי', 'לפני', 'איזה', 'זו', 'תודה', 'הרבה', 'להם', 'פעם', 'אל', 'אותי', 'הכי', 'פה', 'אפשר', 'באמת', 'נראה', 'רוצה', 'יום', 'מאוד', 'שיש', 'בין', 'משהו', 'יכול', 'אולי', 'בכל', 'אנשים', 'אנחנו', 'יודע', 'נכון', 'נגד', 'זאת', 'בלי', 'פשוט', 'הכל', 'שאתה', 'יהיה', 'לנו', 'היו', 'לעשות', 'דבר', 'אתם', 'וגם', 'ככה', 'אותך', 'הייתי', 'בדיוק', 'אדם', 'קצת', 'אותם', 'בן', 'שנים', 'כאן', 'בגלל', 'שהם', 'כזה', 'אומר', 'לה', 'וזה', 'שנה', 'עושה', 'אותה', 'שלהם', 'אחת', 'בכלל', 'עדיין', 'שלנו', 'תמיד', 'ברור', 'ואני', 'לכל', 'אף', 'בו', 'שני', 'אחר', 'בית', 'מישהו', 'חושב', 'אפילו', 'האם', 'מעניין', 'לכם', 'מבין', 'אלא', '

In [8]:
df_train2 = df_train.copy()

In [9]:
# Replacing All F and M with 0 and 1 respectively
df_train.gender.replace({'f':0,'m':1},inplace=True)
df_train.head(8)

,story,gender
0,"בוקר אחד קמתי סהרורי יצאתי מהמיטה קצת מטושטש ,...",1
1,לחבר שלי היה יום הולדת וחיפשנו מה אפשר לעשות ל...,1
2,"השנה האחרונה הייתה שנת קורונה, שנה לא פשוטה בק...",1
3,"לפני כחצי שנה עברתי לגור בצפון עם בת זוגתי, עב...",1
4,"יום חמישי רגיל, תמיד מתחיל לעבור טיפה מאוחר יו...",1
5,בשנה האחרונה חוויתי את מגפת הקורונה שהכריח את ...,1
6,בסמסטר קודם אני וכמה חברים ללימודים קבענו להיפ...,1
7,בשנה האחרונה למרות שלא היו יותר מידיי דברים לע...,0


In [10]:
df_train.gender.unique()

array([1, 0], dtype=int64)

In [11]:
df_train.dtypes

story     object
gender     int64
dtype: object

In [12]:
Xfeatures = df_train['story']
Xfeatures.dtypes

dtype('O')

In [13]:
df_train2.gender.replace({'f':1,'m':0},inplace=True)
df_train2.head(8)

,story,gender
0,"בוקר אחד קמתי סהרורי יצאתי מהמיטה קצת מטושטש ,...",0
1,לחבר שלי היה יום הולדת וחיפשנו מה אפשר לעשות ל...,0
2,"השנה האחרונה הייתה שנת קורונה, שנה לא פשוטה בק...",0
3,"לפני כחצי שנה עברתי לגור בצפון עם בת זוגתי, עב...",0
4,"יום חמישי רגיל, תמיד מתחיל לעבור טיפה מאוחר יו...",0
5,בשנה האחרונה חוויתי את מגפת הקורונה שהכריח את ...,0
6,בסמסטר קודם אני וכמה חברים ללימודים קבענו להיפ...,0
7,בשנה האחרונה למרות שלא היו יותר מידיי דברים לע...,1


### Your implementation:
Write your code solution in the following code-cells

In [14]:
# stopwords - Feature Extraction 
count_vect = CountVectorizer(ngram_range=(1,1),stop_words=list_hebrow_stopwords,token_pattern=r"[א-ת*]+",max_features=10000)
X_train_counts = count_vect.fit_transform(Xfeatures)
X_train_counts.shape

(364, 10000)

In [15]:
X_train_normalized = preprocessing.normalize(X_train_counts, norm='l1')
#X_train_normalized.toarray()
X_train_normalized.toarray()[0:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
print(count_vect.get_feature_names())

['א', 'אבא', 'אבוא', 'אבודים', 'אבטחת', 'אבטלה', 'אבי', 'אביא', 'אבין', 'אבן', 'אגיע', 'אגם', 'אגרסיבית', 'אדווין', 'אדום', 'אדומה', 'אדמה', 'אהבה', 'אהבת', 'אהבתי', 'אהוב', 'אהיה', 'אובר', 'אוגוסט', 'אודות', 'אודם', 'אוהבים', 'אוהבת', 'אוהד', 'אוהדי', 'אוהדים', 'אוהל', 'אוהלים', 'אווי', 'אוויר', 'אווירה', 'אוותר', 'אוזן', 'אוטו', 'אוטובוס', 'אוטובוסים', 'אוטוטו', 'אוטומט', 'אוטומטי', 'אוי', 'אוירה', 'אוכל', 'אוכלים', 'אוכלת', 'אולם', 'אומנה', 'אומנם', 'אומרים', 'אומרת', 'אוניברסלי', 'אונים', 'אונליין', 'אופטימיים', 'אופי', 'אופייני', 'אופן', 'אופנוע', 'אופני', 'אופניים', 'אופציות', 'אוציא', 'אוקטובר', 'אוקטן', 'אוקיי', 'אור', 'אורגניי', 'אורות', 'אורז', 'אורח', 'אורחים', 'אורחת', 'אוריד', 'אורך', 'אורכה', 'אורכו', 'אושוויה', 'אושפז', 'אושפזתי', 'אושר', 'אושרה', 'אות', 'אותוו', 'אותות', 'אותותיה', 'אותיות', 'אותן', 'אותנטי', 'אותתי', 'אזור', 'אזורי', 'אזורים', 'אזכה', 'אזכור', 'אזעקת', 'אזק', 'אזרוק', 'אזרח', 'אזרחות', 'אזרחי', 'אזרחיות', 'אזרחים', 'אח', 'אחאנו', 'אחגוג', 'אחדות', 'אחה

In [17]:
#Vectorization
from sklearn.model_selection import train_test_split

# Features
X_train_counts
# Labels
Y_train_counts = df_train.gender
Y_train_counts2 = df_train2.gender


In [18]:
print(X_train_counts.shape)
print(Y_train_counts.shape)
print(Y_train_counts2.shape)


(364, 10000)
(364,)
(364,)


In [19]:
#f1_male:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X_train_counts, Y_train_counts, test_size=0.33
                                                    #stratify=Y_train_counts 
                                                    , random_state=32)
print(X_train.shape)
print(X_test.shape)

(243, 10000)
(121, 10000)


In [20]:
#Vocabulary
X_train_normalized = count_vect.get_feature_names()
print(X_train_normalized[:10])
print(X_train_normalized[10000:10020])
print(X_train_normalized[::2000])

['א', 'אבא', 'אבוא', 'אבודים', 'אבטחת', 'אבטלה', 'אבי', 'אביא', 'אבין', 'אבן']
[]
['א', 'ההיגיון', 'ולחשוף', 'לעזוב', 'סופרים']


In [21]:
#Classification
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import MultinomialNB

scaler = StandardScaler(with_mean=False)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

clf = MultinomialNB()
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

0.7355371900826446

In [22]:
y_pred = clf.predict(X_test)

#visualize the predictions using metrics classification report
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.47      0.29      0.36        31
           1       0.78      0.89      0.83        90

    accuracy                           0.74       121
   macro avg       0.63      0.59      0.60       121
weighted avg       0.70      0.74      0.71       121



In [23]:
from sklearn.metrics import f1_score

# Accuracy of our Model
print("Accuracy of Model",clf.score(X_test,y_test)*100,"%")

# Accuracy of our Model
print("Accuracy of Model",clf.score(X_train,y_train)*100,"%")

# Accuracy of our f1_score
# f1=2*(precision * recall)/(precision + recall)
# Average_f1=(f1_male+f1_female)/2
print("f1_male 1: ", f1_score(y_test, y_pred,pos_label=1))
print("f1_male 0: ", f1_score(y_test, y_pred,pos_label=0))

f1_male = f1_score(y_test, y_pred, average='micro')
print("f1_male_total: ", f1_male)


Accuracy of Model 73.55371900826447 %
Accuracy of Model 100.0 %
f1_male 1:  0.8333333333333334
f1_male 0:  0.36
f1_male_total:  0.7355371900826446


In [24]:
#f1_female:
# Train Test Split
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train_counts, Y_train_counts2, test_size=0.33
                                                    #stratify=Y_train_counts2 
                                                        , random_state=42)
print(X_train2.shape)
print(X_test2.shape)

(243, 10000)
(121, 10000)


In [25]:
#Classification
from sklearn.linear_model import LogisticRegression

scaler = StandardScaler(with_mean=False)
X_train2 = scaler.fit_transform(X_train2)
X_test2 = scaler.transform(X_test2)

#clf2 = MultinomialNB()
clf2 = LogisticRegression()
clf2.fit(X_train2,y_train2)
clf2.score(X_test2,y_test2)

0.768595041322314

In [26]:
y_pred2 = clf2.predict(X_test2)

#visualize the predictions using metrics classification report
print(metrics.classification_report(y_test2, y_pred2))
print("f1_female 1: ", f1_score(y_test2, y_pred2,pos_label=1))
print("f1_female 0: ", f1_score(y_test2, y_pred2,pos_label=0))

f1_female = f1_score(y_test2, y_pred2, average='micro')
print("f1_female_total: ", f1_female)


              precision    recall  f1-score   support

           0       0.76      1.00      0.87        91
           1       1.00      0.07      0.12        30

    accuracy                           0.77       121
   macro avg       0.88      0.53      0.50       121
weighted avg       0.82      0.77      0.68       121

f1_female 1:  0.125
f1_female 0:  0.8666666666666666
f1_female_total:  0.768595041322314


In [27]:
# Average_f1=(f1_male+f1_female)/2
# Average_f1 = (f1_score(y_test, y_pred,pos_label=1)+f1_score(y_test, y_pred,pos_label=0))/2

In [28]:
Average_f1=(f1_male+f1_female)/2
print(Average_f1)

0.7520661157024793


In [38]:
# YOUR CODE HERE
# predicted_category Save output
df_copy = df_test.copy()
df_copy.head(8)

,test_example_id,story
0,0,כחלק ממסגרת ההתנדבות שלי במגלה אני הולך לפעמיי...
1,1,"לפני שנה החלטתי שאני רוצה להיות טייס, התחלתי ל..."
2,2,"בתקופת הקורונה של תחילת החיסונים נגד קורונה, א..."
3,3,כפי שכולם מכירים או שמעו מחברים עולם הדייטים ה...
4,4,"אחת החוויות שהכי זכורות לי, זו החוויה בפרו בטי..."
5,5,"ראיתי בקורונה את עולמי נופל בין ידי , בת זוגתי..."
6,6,"באחת תקופות המבחנים שהיו השנה , קראתי לא נכון ..."
7,7,"כשהשתחררתי מהצבא בגיל 22 ישר התחלתי לעבוד, הצב..."


In [39]:
import random
list = [0,1]
df_copy['predicted_category'] =  np.random.choice(list, size=len(df_copy))
df_copy.head(8)

,test_example_id,story,predicted_category
0,0,כחלק ממסגרת ההתנדבות שלי במגלה אני הולך לפעמיי...,0
1,1,"לפני שנה החלטתי שאני רוצה להיות טייס, התחלתי ל...",0
2,2,"בתקופת הקורונה של תחילת החיסונים נגד קורונה, א...",0
3,3,כפי שכולם מכירים או שמעו מחברים עולם הדייטים ה...,1
4,4,"אחת החוויות שהכי זכורות לי, זו החוויה בפרו בטי...",0
5,5,"ראיתי בקורונה את עולמי נופל בין ידי , בת זוגתי...",1
6,6,"באחת תקופות המבחנים שהיו השנה , קראתי לא נכון ...",1
7,7,"כשהשתחררתי מהצבא בגיל 22 ישר התחלתי לעבוד, הצב...",0


In [40]:
# YOUR CODE HERE
# A function to do it

df_predicted = df_copy.copy()
df_predicted.drop(['story',], axis=1, inplace=True)
df_predicted.head(8)

,test_example_id,predicted_category
0,0,0
1,1,0
2,2,0
3,3,1
4,4,0
5,5,1
6,6,1
7,7,0


In [41]:
df_predicted.predicted_category.replace({0:'f',1:'m'},inplace=True)
df_predicted.head(8)

,test_example_id,predicted_category
0,0,f
1,1,f
2,2,f
3,3,m
4,4,f
5,5,m
6,6,m
7,7,f


In [ ]:
# Add as many code cells as you need
# YOUR CODE HERE

#predicted_col = clf.predict(df_copy['story'])
#drop = df_copy.copy().drop(['story'], axis = 1)
#df_predicted = pd.DataFrame(data = {"predicted_category": predicted_col})
#df_predicted["predicted_category"].replace({0:"m",1:"f"}, inplace = True)

### Save output to csv
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [42]:
df_predicted.to_csv('classification_results.csv',index=False)